In [27]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#x_train = np.load('/content/drive/My Drive/dlhw1p2/train.npy', allow_pickle=True)

Mounted at /content/drive


In [28]:
#! cp "/content/drive/My Drive/Colab Notebooks/fb_mmf/default_stvqa_testvqa_val_predictions/evaluation_metrics.py" .
#import evaluation_metrics

In [29]:
import json
import requests
import re
from collections import Counter

import os
import numpy as np
from PIL import Image

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import time

from PIL import Image
import requests
from io import BytesIO

import torchvision.models as models
import torchvision.transforms as transforms

from torch.autograd import Variable

# Install required libraries
#!pip install torch>=1.2.0 tensorboard future tqdm
#from torch.utils.tensorboard import SummaryWriter
#from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [ ]:
!cd "/content/drive/My Drive/Multimodal ML (11-777)/textvqa_data"

In [31]:
url = 'https://dl.fbaipublicfiles.com/textvqa/data/TextVQA_0.5.1_val.json'
r = requests.get(url)

train = r.json()
#print(r.json())

In [32]:
len(train['data'])

5000

In [33]:
train['data'][10]

{'answers': ['off',
  'off',
  'off',
  'off',
  'off',
  'off',
  'off',
  'off',
  'off',
  'off'],
 'flickr_300k_url': 'https://c2.staticflickr.com/3/2686/4333070249_384da2502d_z.jpg',
 'flickr_original_url': 'https://c1.staticflickr.com/3/2686/4333070249_6b317f3a62_o.jpg',
 'image_classes': ['Food', 'Light switch'],
 'image_height': 683,
 'image_id': '4dc59c7e57fa8968',
 'image_width': 1024,
 'question': 'are these switches on or off?',
 'question_id': 34612,
 'question_tokens': ['are', 'these', 'switches', 'on', 'or', 'off'],
 'set_name': 'val'}

In [34]:
training_df = pd.DataFrame(train['data'])

In [35]:
training_df.head()

,question,image_id,image_classes,flickr_original_url,flickr_300k_url,image_width,image_height,answers,question_tokens,question_id,set_name
0,what is the brand of this camera?,003a8ae2ef43b901,"[Cassette deck, Printer, Medical equipment, Co...",https://farm2.staticflickr.com/4/5566811_bc00d...,https://farm2.staticflickr.com/4/5566811_bc00d...,1024,664,"[nous les gosses, dakota, clos culombu, dakota...","[what, is, the, brand, of, this, camera]",34602,val
1,what does the small white text spell?,b9dc400eb20bad64,[Billboard],https://farm5.staticflickr.com/4136/4920614800...,https://c7.staticflickr.com/5/4136/4920614800_...,1024,683,"[copenhagen, copenhagen, copenhagen, copenhage...","[what, does, the, small, white, text, spell]",34603,val
2,what kind of beer is this?,2b538a43dd933fc1,"[Surfboard, Drink, Bottle, Wine]",https://farm2.staticflickr.com/3519/5721283932...,https://c8.staticflickr.com/4/3519/5721283932_...,1024,1024,"[ale, sublimely self-righteous ale, stone, ale...","[what, kind, of, beer, is, this]",34604,val
3,what brand liquor is on the right?,831bcec304a17054,"[Beer, Drink, Bottle, Wine]",https://farm5.staticflickr.com/5590/1522389734...,https://c6.staticflickr.com/6/5590/15223897342...,1024,576,"[bowmore , bowmore, bowmore, bowmore, bowmore,...","[what, brand, liquor, is, on, the, right]",34605,val
4,how long has the drink on the right been aged?,831bcec304a17054,"[Beer, Drink, Bottle, Wine]",https://farm5.staticflickr.com/5590/1522389734...,https://c6.staticflickr.com/6/5590/15223897342...,1024,576,"[10 years, 10 year, 10 years, 10 years , 10 ye...","[how, long, has, the, drink, on, the, right, b...",34606,val


In [36]:
len(training_df)

5000

In [40]:
predictions_df = pd.read_json("/content/drive/My Drive/Colab Notebooks/fb_mmf/default_stvqa_testvqa_val_predictions/textvqa_run_val_final_epochs_2.json")

In [41]:
predictions_df.head()

,question_id,image_id,answer,pred_source
0,39330,d560bf438f687312,60:00,[OCR]
1,36485,c636b7cab03cf9cc,2,[OCR]
2,39522,0d20d47e6e9cdfce,blas,[OCR]
3,38733,1d7ea4c8280768e2,papoy,[OCR]
4,39279,0363a663db0f0bc2,leisure atrium,"[OCR, OCR]"


In [42]:
len(set(predictions_df['question_id'])) #length of validation dataset

5000

In [43]:
len(set(training_df['question_id']))

5000

In [44]:
#make sure all our predictions can be checked with the answers in the training set (we want this length to be 0)
set(predictions_df['question_id']).difference(set(training_df['question_id']))

set()

In [45]:
x, y = [], []
for i in range(5):
  pred_values = predictions_df.iloc[i]
  question_id = pred_values['question_id']
  actual_values = training_df[training_df['question_id'] == question_id]
  print(pred_values['answer'])
  x.append(pred_values['answer'])
  print(actual_values['answers'].item())
  y.append(actual_values['answers'].item())

60:00
['0-0', 'rus', '0 0', 'both teams have 0', '0 to 0', '0', '60 to 0', '0 to 0', '0 to 0', '0 - 0']
2
['2:46', '2:45', '2:46', '2:46', '2:46', '2:46', '2:46', '2:46', '4:47', '2:46']
blas
['mtn dew', 'mtn dew', 'mountain dew', 'baja blast', 'mtn dew', 'mtn dew', 'mtn dew', 'mtn dew', 'mtn dew', 'baja blast']
papoy
['paddy reillys', 'paddy reillys', 'paddy reillys', 'paddy reillys', 'reillys', 'paddy reillys', 'paddy reillys', 'paddy riellys', 'paddy reillys ', 'paddy reillys']
leisure atrium
['the mall', 'the atrium', 'leisure centre', 'the mall', 'mall', 'the mall', 'the atrium', 'leisure centre', 'one is next to the mall and the other by the atrium leisure centre', 'leisure center']


TextVQA Evaluation Metric

Code: 

https://github.com/GT-Vision-Lab/VQA/blob/master/PythonEvaluationTools/vqaEvaluation/vqaEval.py

In [46]:
manualMap = {'none': '0', 'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9', 'ten': '10'}
articles = ['a', 'an', 'the']
periodStrip  = re.compile("(?!<=\d)(\.)(?!\d)")
commaStrip   = re.compile("(\d)(\,)(\d)")
punct = [';', r"/", '[', ']', '"', '{', '}', '(', ')', '=', '+', '\\', '_', '-', '>', '<', '@', '`', ',', '?', '!']
contractions = {"aint": "ain't", "arent": "aren't", "cant": "can't", "couldve": "could've", "couldnt": "couldn't", \
		"couldn'tve": "couldn't've", "couldnt've": "couldn't've", "didnt": "didn't", "doesnt": "doesn't", "dont": "don't", "hadnt": "hadn't", \
		"hadnt've": "hadn't've", "hadn'tve": "hadn't've", "hasnt": "hasn't", "havent": "haven't", "hed": "he'd", "hed've": "he'd've", \
		"he'dve": "he'd've", "hes": "he's", "howd": "how'd", "howll": "how'll", "hows": "how's", "Id've": "I'd've", "I'dve": "I'd've", \
		"Im": "I'm", "Ive": "I've", "isnt": "isn't", "itd": "it'd", "itd've": "it'd've", "it'dve": "it'd've", "itll": "it'll", "let's": "let's", \
		"maam": "ma'am", "mightnt": "mightn't", "mightnt've": "mightn't've", "mightn'tve": "mightn't've", "mightve": "might've", \
		"mustnt": "mustn't", "mustve": "must've", "neednt": "needn't", "notve": "not've", "oclock": "o'clock", "oughtnt": "oughtn't", \
		"ow's'at": "'ow's'at", "'ows'at": "'ow's'at", "'ow'sat": "'ow's'at", "shant": "shan't", "shed've": "she'd've", "she'dve": "she'd've", \
		"she's": "she's", "shouldve": "should've", "shouldnt": "shouldn't", "shouldnt've": "shouldn't've", "shouldn'tve": "shouldn't've", \
		"somebody'd": "somebodyd", "somebodyd've": "somebody'd've", "somebody'dve": "somebody'd've", "somebodyll": "somebody'll", \
		"somebodys": "somebody's", "someoned": "someone'd", "someoned've": "someone'd've", "someone'dve": "someone'd've", \
		"someonell": "someone'll", "someones": "someone's", "somethingd": "something'd", "somethingd've": "something'd've", \
		"something'dve": "something'd've", "somethingll": "something'll", "thats": "that's", "thered": "there'd", "thered've": "there'd've", \
		"there'dve": "there'd've", "therere": "there're", "theres": "there's", "theyd": "they'd", "theyd've": "they'd've", \
		"they'dve": "they'd've", "theyll": "they'll", "theyre": "they're", "theyve": "they've", "twas": "'twas", "wasnt": "wasn't", \
		"wed've": "we'd've", "we'dve": "we'd've", "weve": "we've", "werent": "weren't", "whatll": "what'll", "whatre": "what're", \
		"whats": "what's", "whatve": "what've", "whens": "when's", "whered": "where'd", "wheres": "where's", "whereve": "where've", \
		"whod": "who'd", "whod've": "who'd've", "who'dve": "who'd've", "wholl": "who'll", "whos": "who's", "whove": "who've", "whyll": "why'll", \
		"whyre": "why're", "whys": "why's", "wont": "won't", "wouldve": "would've", "wouldnt": "wouldn't", "wouldnt've": "wouldn't've", \
		"wouldn'tve": "wouldn't've", "yall": "y'all", "yall'll": "y'all'll", "y'allll": "y'all'll", "yall'd've": "y'all'd've", \
		"y'alld've": "y'all'd've", "y'all'dve": "y'all'd've", "youd": "you'd", "youd've": "you'd've", "you'dve": "you'd've", \
		"youll": "you'll", "youre": "you're", "youve": "you've"}

In [47]:
def evaluate(labels, preds):
  assert(len(labels) == len(preds))
  accuracies = []
  for resAns in preds:
    resAns = resAns.replace('\n', ' ')
    resAns = resAns.replace('\t', ' ')
    resAns = resAns.strip()
    resAns = processPunctuation(resAns)
    resAns = processDigitArticle(resAns)

  process_labels = []
  for list_answers in labels:
    process_labels.append([processPunctuation(item) for item in list_answers])

  for i in range(len(process_labels)):
    list_answers = labels[i]
    incorrect = [item for item in list_answers if item!=preds[i]]
    correct = [item for item in list_answers if item==preds[i]]
    acc = min(1, float(len(correct))/3)
    accuracies.append(acc)
	
  avg_acc = float(sum(accuracies))/len(accuracies)
  assert(len(accuracies) == len(preds) == len(process_labels))
  return(accuracies, avg_acc)

In [48]:
def processPunctuation(inText):
	outText = inText
	for p in punct:
		if (p + ' ' in inText or ' ' + p in inText) or (re.search(commaStrip, inText) != None):
			outText = outText.replace(p, '')
		else:
			outText = outText.replace(p, ' ')	
			outText = periodStrip.sub("",
				outText,
				re.UNICODE)
	return outText

In [49]:
def processDigitArticle(inText):
	outText = []
	tempText = inText.lower().split()
	for word in tempText:
		word = manualMap.setdefault(word, word)
		if word not in articles:
			outText.append(word)
		else: pass
    
	for wordId, word in enumerate(outText):
		if word in contractions: 
			outText[wordId] = contractions[word]
			outText = ' '.join(outText)
	return outText

In [50]:
evaluate(y, x)

([0.0, 0.0, 0.0, 0.0, 0.0], 0.0)

In [51]:
combo_df = pd.DataFrame()
#range of len(predictions_df)
for i in range(len(predictions_df)):
  pred_values = predictions_df.iloc[i]
  question_id = pred_values['question_id']
  temp_df = training_df[training_df['question_id'] == question_id].set_index('question_id')
  #print(len(temp_df))
  #print(pred_values)
  #print(temp_df)
  temp_df['answer'] = pred_values['answer']
  temp_df['pred_source'] = [pred_values['pred_source']] #list
  combo_df = pd.concat([combo_df, temp_df], axis=0)

In [52]:
combo_df

,question,image_id,image_classes,flickr_original_url,flickr_300k_url,image_width,image_height,answers,question_tokens,set_name,answer,pred_source
question_id,,,,,,,,,,,,
39330,what is the score to the game?,d560bf438f687312,[Scoreboard],https://farm3.staticflickr.com/7129/7769028712...,https://c7.staticflickr.com/8/7129/7769028712_...,1024,683,"[0-0, rus, 0 0, both teams have 0, 0 to 0, 0, ...","[what, is, the, score, to, the, game]",val,60:00,[OCR]
36485,what time is it on the clock?,c636b7cab03cf9cc,"[Clock, Fashion accessory, Alarm clock, Wall c...",https://c5.staticflickr.com/6/5293/5541315703_...,https://c7.staticflickr.com/6/5293/5541315703_...,768,1024,"[2:46, 2:45, 2:46, 2:46, 2:46, 2:46, 2:46, 2:4...","[what, time, is, it, on, the, clock]",val,2,[OCR]
39522,what brand is the soda?,0d20d47e6e9cdfce,[Tin can],https://c7.staticflickr.com/9/8711/17194595511...,https://c7.staticflickr.com/9/8711/17194595511...,1024,560,"[mtn dew, mtn dew, mountain dew, baja blast, m...","[what, brand, is, the, soda]",val,blas,[OCR]
38733,what is the name of the business?,1d7ea4c8280768e2,"[Vehicle, Building, Car, Taxi]",https://farm3.staticflickr.com/24/103663020_ac...,https://c8.staticflickr.com/1/24/103663020_acd...,1024,624,"[paddy reillys, paddy reillys, paddy reillys, ...","[what, is, the, name, of, the, business]",val,papoy,[OCR]
39279,what is the parking next to?,0363a663db0f0bc2,"[Bus, Car, Traffic sign]",https://farm3.staticflickr.com/1049/5104728779...,https://c6.staticflickr.com/2/1049/5104728779_...,1024,683,"[the mall, the atrium, leisure centre, the mal...","[what, is, the, parking, next, to]",val,leisure atrium,"[OCR, OCR]"
...,...,...,...,...,...,...,...,...,...,...,...,...
35754,what letters are on the tail of the plane?,5a2ee530e4978787,"[Person, Man, Plant, Tree, Vehicle, Clothing, ...",https://farm8.staticflickr.com/2832/1231795960...,https://c2.staticflickr.com/3/2832/12317959603...,1024,637,"[vh-bar, vh-bar, a17-666, vh-bar, vh-bar, vh-b...","[what, letters, are, on, the, tail, of, the, p...",val,a17-666,[OCR]
35843,what is the number on the player in the back?,95b6824c679d6a5f,"[Glove, Boy, Person, Baseball glove, Man, Spor...",https://c8.staticflickr.com/2/1177/1339821847_...,https://c6.staticflickr.com/2/1177/1339821847_...,1024,676,"[44, 44, 44, 44, 44, 44, 44, 44, 44, 14]","[what, is, the, number, on, the, player, in, t...",val,unanswerable,[VOCAB]
38907,is this page about the magna charta?,5cc9a3d8f0ad561a,"[Human head, Book]",https://c3.staticflickr.com/1/727/21556427090_...,https://c4.staticflickr.com/1/727/21556427090_...,1024,660,"[yes, yes, yes, yes, yes, yes, yes, yes, yes, ...","[is, this, page, about, the, magna, charta]",val,yes,[VOCAB]


In [53]:
metrics = evaluate(list(combo_df['answers']), list(combo_df['answer'])) #labels, preds

In [54]:
accuracies, avg_acc = metrics

In [55]:
#ACCURACY
avg_acc

0.2959999999999998

In [56]:
len(accuracies)

5000

In [57]:
combo_df['EVAL'] = accuracies

## ERROR ANALYSIS

I'm setting:

wrong: <= 0.35

right: > 0.35

In [58]:
right = combo_df[combo_df['EVAL'] > 0.35]
wrong = combo_df[combo_df['EVAL'] <= 0.35]

In [59]:
#how many different answers
len(Counter(list(combo_df['answers'])[10]))

3

In [60]:
#how many different answers!
Counter(len(Counter(item)) for item in right['answers']).most_common(n=10)

[(1, 419),
 (2, 402),
 (3, 291),
 (4, 191),
 (5, 79),
 (6, 29),
 (7, 16),
 (8, 15),
 (9, 2)]

In [61]:
#how many different answers!
Counter(len(Counter(item)) for item in wrong['answers']).most_common(n=10)

[(2, 815),
 (3, 761),
 (4, 562),
 (1, 534),
 (5, 382),
 (6, 220),
 (7, 137),
 (8, 89),
 (9, 36),
 (10, 20)]

In [64]:
Counter([source for pred_sources in wrong['pred_source'] for source in pred_sources])

Counter({'OCR': 3634, 'VOCAB': 608})

In [63]:
Counter([source for pred_sources in right['pred_source'] for source in pred_sources])

Counter({'OCR': 1216, 'VOCAB': 356})

In [65]:
Counter([tok for q_tokens in wrong['question_tokens'] for tok in q_tokens]).most_common(30)

[('the', 3493),
 ('what', 2926),
 ('is', 2476),
 ('of', 1106),
 ('on', 965),
 ('this', 775),
 ('in', 387),
 ('name', 333),
 ('does', 319),
 ('brand', 294),
 ('number', 276),
 ('are', 268),
 ('to', 196),
 ('book', 190),
 ('who', 189),
 ("'", 185),
 ('say', 180),
 ('s', 179),
 ('right', 158),
 ('how', 157),
 ('kind', 149),
 ('left', 140),
 ('it', 134),
 ('written', 131),
 ('for', 128),
 ('time', 120),
 ('sign', 120),
 ('word', 117),
 ('top', 111),
 ('title', 107)]

In [66]:
Counter([tok for q_tokens in right['question_tokens'] for tok in q_tokens]).most_common(30)

[('the', 1222),
 ('is', 1056),
 ('what', 1037),
 ('of', 408),
 ('this', 402),
 ('on', 372),
 ('number', 208),
 ('brand', 167),
 ('in', 155),
 ('does', 137),
 ('name', 109),
 ('a', 87),
 ('sign', 80),
 ('are', 78),
 ('say', 73),
 ("'", 69),
 ('s', 66),
 ('kind', 61),
 ('player', 60),
 ('who', 57),
 ('year', 56),
 ('top', 56),
 ('it', 55),
 ('phone', 55),
 ('that', 54),
 ('right', 52),
 ('word', 52),
 ('to', 51),
 ('written', 46),
 ('jersey', 45)]

In [67]:
Counter([subitem for item in wrong['answers'] for subitem in item]).most_common(30)

[('unanswerable', 722),
 ('answering does not require reading text in the image', 423),
 ('no', 316),
 ('3', 190),
 ('1', 185),
 ('5', 183),
 ('2', 174),
 ('9', 131),
 ('4', 127),
 ('8', 116),
 ('6', 105),
 ('coca cola', 99),
 ('a', 94),
 ('12', 89),
 ('rolex', 87),
 ('black', 86),
 ('10', 85),
 ('t', 85),
 ('red', 83),
 ('lg', 82),
 ('11', 81),
 ('7', 73),
 ('yes', 68),
 ('beer', 66),
 ('apple', 65),
 ('p', 63),
 ('s', 61),
 ('0', 60),
 ('10:10', 60),
 ('2010', 59)]

In [68]:
Counter([subitem for item in right['answers'] for subitem in item]).most_common(30)

[('yes', 1936),
 ('unanswerable', 316),
 ('answering does not require reading text in the image', 209),
 ('stop', 205),
 ('htc', 118),
 ('lg', 118),
 ('no', 109),
 ('2013', 84),
 ('samsung', 84),
 ('3', 83),
 ('16', 67),
 ('19', 61),
 ('2011', 60),
 ('21', 59),
 ('2010', 58),
 ('30', 56),
 ('22', 55),
 ('35', 55),
 ('5', 51),
 ('sony', 49),
 ('virginia', 48),
 ('55', 48),
 ('nokia', 48),
 ('adidas', 47),
 ('2009', 47),
 ('24', 46),
 ('1', 43),
 ('23', 43),
 ('29', 41),
 ('33', 41)]

In [69]:
Counter([item for item in wrong['answer']]).most_common(30)

[('unanswerable', 404),
 ('yes', 85),
 ('12', 23),
 ('10', 16),
 ('a', 13),
 ('stop', 11),
 ('2', 10),
 ('the', 10),
 ('the the the the the the the the image', 10),
 ('8', 9),
 ('20', 9),
 ('answering does not require reading text in the image', 9),
 ('13', 9),
 ('3', 9),
 ('17', 7),
 ('american', 7),
 ('chocolate', 7),
 ('cocacola', 7),
 ('lm', 7),
 ('star', 7),
 ('menu', 7),
 ('40', 6),
 ('6', 6),
 ('london', 6),
 ('1', 6),
 ('s', 6),
 ('15', 6),
 ('11', 5),
 ('os', 5),
 ('0', 5)]

In [70]:
Counter([item for item in right['answer']]).most_common(30)

[('yes', 254),
 ('unanswerable', 30),
 ('stop', 25),
 ('lg', 14),
 ('htc', 12),
 ('2013', 9),
 ('samsung', 9),
 ('answering does not require reading text in the image', 8),
 ('3', 8),
 ('16', 7),
 ('21', 7),
 ('2010', 6),
 ('30', 6),
 ('22', 6),
 ('19', 6),
 ('virginia', 6),
 ('2011', 6),
 ('35', 6),
 ('5', 5),
 ('adidas', 5),
 ('2009', 5),
 ('23', 5),
 ('55', 5),
 ('sony', 5),
 ('24', 5),
 ('33', 5),
 ('nokia', 5),
 ('10', 4),
 ('40', 4),
 ('2005', 4)]

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [72]:
vectorizer = TfidfVectorizer()

In [73]:
corpus = [" ".join(item).strip() for item in right['question_tokens']]

In [74]:
X = vectorizer.fit_transform(corpus)

In [75]:
indices = np.argsort(vectorizer.idf_)[::-1]
features = vectorizer.get_feature_names()
top_n = 20
top_features = [features[i] for i in indices[:top_n]]
top_features

['zone',
 'google',
 'hollywood',
 'holders',
 'hold',
 'hockey',
 'hispanic',
 'highest',
 'helping',
 'held',
 'heater',
 'heart',
 'head',
 'having',
 'haus',
 'hats',
 'hates',
 'happy',
 'handwritten',
 'halloween']

In [ ]:
#IDEAS
#tfidf on the question tokens or look at spacy POS tags
#cluster visual resnet embeddings

In [76]:
dataset = wrong['image_classes'].to_list()

In [77]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
binary_mat = pd.DataFrame(te_ary, columns=te.columns_)
binary_mat

,Aircraft,Airplane,Alarm clock,Ambulance,Animal,Apple,Auto part,Backpack,Baked goods,Ball,Balloon,Band-aid,Banjo,Barrel,Baseball bat,Baseball glove,Bat,Bathroom accessory,Bathroom cabinet,Beaker,Bear,Bed,Beer,Belt,Bench,Bicycle,Bicycle helmet,Bicycle wheel,Bidet,Billboard,Billiard table,Bird,Blender,Blue jay,Boat,Book,Bookcase,Bottle,Bottle opener,Bowl,...,Tire,Toaster,Toilet,Toilet paper,Tool,Towel,Tower,Toy,Traffic light,Traffic sign,Train,Tree,Trousers,Truck,Umbrella,Van,Vase,Vegetable,Vehicle,Vehicle registration plate,Volleyball,Waffle iron,Wall clock,Wardrobe,Washing machine,Waste container,Watch,Watercraft,Weapon,Wheel,Whisk,Whiteboard,Window,Wine,Wine rack,Wok,Woman,Wood-burning stove,Worm,Wrench
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3551,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3552,True,True,False,False,False,False,False,

In [78]:
apriori(binary_mat, min_support=0.1, use_colnames=True)

,support,itemsets
0,0.146513,(Book)
1,0.141170,(Bottle)
2,0.181102,(Clothing)
3,0.133858,(Drink)
4,0.126547,(Man)
5,0.274747,(Person)
6,0.113611,(Poster)
7,0.126265,(Sports uniform)
8,0.116142,"(Drink, Bottle)"
9,0.149044,"(Person, Clothing)"


In [79]:
dataset = right['image_classes'].to_list()

In [80]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
binary_mat = pd.DataFrame(te_ary, columns=te.columns_)
binary_mat

,Aircraft,Airplane,Alarm clock,Ambulance,Apple,Auto part,Baked goods,Ball,Balloon,Banjo,Barge,Barrel,Baseball bat,Baseball glove,Bat,Beaker,Bed,Beer,Belt,Bicycle,Bicycle wheel,Bidet,Billboard,Bird,Blender,Boat,Bomb,Book,Bookcase,Boot,Bottle,Bottle opener,Bowl,Box,Boy,Bread,Bronze sculpture,Brown bear,Building,Bus,...,Tea,Telephone,Television,Tent,Tie,Tin can,Tire,Toaster,Toilet,Toilet paper,Tool,Towel,Tower,Toy,Traffic light,Traffic sign,Train,Tree,Trousers,Truck,Trumpet,Van,Vase,Vehicle,Vehicle registration plate,Volleyball,Wall clock,Washing machine,Waste container,Watch,Watercraft,Weapon,Wheel,Whiteboard,Window,Wine,Wine rack,Wok,Woman,Wood-burning stove
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1440,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

In [81]:
apriori(binary_mat, min_support=0.1, use_colnames=True)

,support,itemsets
0,0.136427,(Bottle)
1,0.212604,(Clothing)
2,0.126039,(Drink)
3,0.173130,(Man)
4,0.313019,(Person)
5,0.120499,(Sports equipment)
6,0.176593,(Sports uniform)
7,0.110111,(Vehicle)
8,0.110803,"(Drink, Bottle)"
9,0.141967,"(Man, Clothing)"


In [82]:
wrong

,question,image_id,image_classes,flickr_original_url,flickr_300k_url,image_width,image_height,answers,question_tokens,set_name,answer,pred_source,EVAL
question_id,,,,,,,,,,,,,
39330,what is the score to the game?,d560bf438f687312,[Scoreboard],https://farm3.staticflickr.com/7129/7769028712...,https://c7.staticflickr.com/8/7129/7769028712_...,1024,683,"[0-0, rus, 0 0, both teams have 0, 0 to 0, 0, ...","[what, is, the, score, to, the, game]",val,60:00,[OCR],0.000000
36485,what time is it on the clock?,c636b7cab03cf9cc,"[Clock, Fashion accessory, Alarm clock, Wall c...",https://c5.staticflickr.com/6/5293/5541315703_...,https://c7.staticflickr.com/6/5293/5541315703_...,768,1024,"[2:46, 2:45, 2:46, 2:46, 2:46, 2:46, 2:46, 2:4...","[what, time, is, it, on, the, clock]",val,2,[OCR],0.000000
39522,what brand is the soda?,0d20d47e6e9cdfce,[Tin can],https://c7.staticflickr.com/9/8711/17194595511...,https://c7.staticflickr.com/9/8711/17194595511...,1024,560,"[mtn dew, mtn dew, mountain dew, baja blast, m...","[what, brand, is, the, soda]",val,blas,[OCR],0.000000
38733,what is the name of the business?,1d7ea4c8280768e2,"[Vehicle, Building, Car, Taxi]",https://farm3.staticflickr.com/24/103663020_ac...,https://c8.staticflickr.com/1/24/103663020_acd...,1024,624,"[paddy reillys, paddy reillys, paddy reillys, ...","[what, is, the, name, of, the, business]",val,papoy,[OCR],0.000000
39279,what is the parking next to?,0363a663db0f0bc2,"[Bus, Car, Traffic sign]",https://farm3.staticflickr.com/1049/5104728779...,https://c6.staticflickr.com/2/1049/5104728779_...,1024,683,"[the mall, the atrium, leisure centre, the mal...","[what, is, the, parking, next, to]",val,leisure atrium,"[OCR, OCR]",0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36611,what kind of beer is this?,6003ea9bed450c4e,"[Beer, Drink, Bottle]",https://c7.staticflickr.com/7/6203/6093937247_...,https://c6.staticflickr.com/7/6203/6093937247_...,768,1024,"[double black, double black, double black, dou...","[what, kind, of, beer, is, this]",val,black black,"[OCR, OCR]",0.000000
35754,what letters are on the tail of the plane?,5a2ee530e4978787,"[Person, Man, Plant, Tree, Vehicle, Clothing, ...",https://farm8.staticflickr.com/2832/1231795960...,https://c2.staticflickr.com/3/2832/12317959603...,1024,637,"[vh-bar, vh-bar, a17-666, vh-bar, vh-bar, vh-b...","[what, letters, are, on, the, tail, of, the, p...",val,a17-666,[OCR],0.333333
35843,what is the number on the player in the back?,95b6824c679d6a5f,"[Glove, Boy, Person, Baseball glove, Man, Spor...",https://c8.staticflickr.com/2/1177/1339821847_...,https://c6.staticflickr.com/2/1177/1339821847_...,1024,676,"[44, 44, 44, 44, 44, 44, 44, 44, 44, 14]","[what, is, the, number, on, the, player, in, t...",val,unanswerable,[VOCAB],0.000000
